In [4]:
import os
import cv2
import numpy as np
from imutils import paths
import pyttsx3
speaker=pyttsx3.init()

In [5]:
from sklearn.preprocessing import LabelBinarizer

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
from keras.preprocessing.image import ImageDataGenerator

In [8]:
datapath=r"C:\Users\krssu\Desktop\7th Sem\DSML\pg\data"
outputmodel=r"C:\Users\krssu\Desktop\7th Sem\DSML\pg\video_classification_model\videoclassificationnmodel"
outputlabelbinarizer=r"C:\Users\krssu\Desktop\7th Sem\DSML\pg\model\videoclassificationbinarizer"
epoch=25

In [9]:
Sports_Labels=set(["basketball"])
speaker.say("Images being loaded......")
print("Images being loaded......")
speaker.runAndWait() 
pathToImages=list(paths.list_images(datapath))
data=[]
labels=[]
for images in pathToImages:
    label=images.split(os.path.sep)[-2]
    if label not in Sports_Labels:
        continue
    image=cv2.imread(images)
    image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    image=cv2.resize(image,(224,224))
    data.append(image)
    labels.append(label)

Images being loaded......


In [10]:
data=np.array(data)
labels=np.array(labels)
lb=LabelBinarizer()
labels=lb.fit_transform(labels) 

In [11]:
(X_train,X_test,Y_train,Y_test)=train_test_split(data,labels,test_size=0.25,stratify=labels,random_state=42)

In [12]:
traininAugmentation=ImageDataGenerator(
    rotation_range=30,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest"
)
validationAugmentation=ImageDataGenerator()
mean=np.array([123.68,116.779,103.939],dtype="float32")
traininAugmentation.mean=mean
validationAugmentation.mean=mean

In [13]:
from keras.applications import ResNet50
from keras.layers import Input
from keras.layers.pooling import AveragePooling2D
from keras.layers.core import Flatten
from keras.layers.core import Dense

from keras.layers.core import Dropout

from keras.models import Model

In [14]:
baseModel=ResNet50(weights="imagenet",include_top=False,input_tensor=Input(shape=(224,224,3)))
headModel=baseModel.output
headModel=AveragePooling2D(pool_size=(7,7))(headModel)
headModel=Flatten(name="flatten")(headModel)
headModel=Dense(512,activation="relu")(headModel)
headModel=Dropout(0.5)(headModel)
headModel=Dense(len(lb.classes_),activation="softmax")(headModel)
model=Model(inputs=baseModel.input,outputs=headModel)
for basemodelLayers in baseModel.layers:
    basemodelLayers.trainable=False

In [15]:
from keras.optimizers import SGD

In [16]:
opt=SGD(lr=0.0001,momentum=0.9,decay=1e-4/epoch)

In [17]:
model.compile(loss="categorical_crossentropy",optimizer=opt,metrics=["accuracy"])

In [19]:
History=model.fit_generator(
traininAugmentation.flow(X_train,Y_train,batch_size=32),
steps_per_epoch=len(X_train)//32,
validation_data=validationAugmentation.flow(X_test,Y_test),
validation_steps=len(X_test)//32,
epochs=epoch
)

Epoch 1/25
11/11 [==============================] - 50s 5s/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/25
11/11 [==============================] - 50s 5s/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 3/25
11/11 [==============================] - 49s 5s/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 4/25
11/11 [==============================] - 53s 5s/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 5/25
11/11 [==============================] - 51s 5s/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 6/25
11/11 [==============================] - 60s 6s/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 7/25
11/11 [==============================] - 67s 6s/step 

In [20]:
import pickle

In [22]:
model.save(outputmodel)
lbinarizer=open(r"C:\Users\krssu\Desktop\7th Sem\DSML\pg\model\videoclassificationbinarizer.pickle","wb")
lbinarizer.write(pickle.dumps(lb))
lbinarizer.close()

INFO:tensorflow:Assets written to: C:\Users\krssu\Desktop\7th Sem\DSML\pg\video_classification_model\videoclassificationnmodel\assets
